### 🛠️ 1. Initialize notebook variables

Configures everything that's needed for deployment. 

👉 **Modify entries under _1) User-defined parameters_ and _3) Define the APIs and their operations and policies_**.

In [ ]:
import utils
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 1
deployment  = INFRASTRUCTURE.SIMPLE_APIM
tags        = ['authX-pro', 'jwt', 'policy-fragment']       # ENTER DESCRIPTIVE TAG(S)
api_prefix  = 'authX-pro-'               # OPTIONAL: ENTER A PREFIX FOR THE APIS TO REDUCE COLLISION POTENTIAL WITH OTHER SAMPLES

# 2) Service-defined parameters (please do not change these)
rg_name       = utils.get_infra_rg_name(deployment, index)
sample_folder = "authX-pro"
nb_helper     = utils.NotebookHelper(sample_folder, rg_name, rg_location, deployment, [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA, INFRASTRUCTURE.SIMPLE_APIM], True)

# 3) Set up the named values
nvs: List[NamedValue] = [
    NamedValue(nb_helper.jwt_key_name, nb_helper.jwt_key_value_bytes_b64, True),
    NamedValue('HRMemberRoleId', Role.HR_MEMBER),
    NamedValue('HRAssociateRoleId', Role.HR_ASSOCIATE),
    NamedValue('HRAdministratorRoleId', Role.HR_ADMINISTRATOR)
]

# 4) Set up the policy fragments
pf_authx_hr_member_xml = utils.read_policy_xml('pf-authx-hr-member.xml', {
    'jwt_signing_key': nb_helper.jwt_key_name,
    'hr_member_role_id': 'HRMemberRoleId'
}, sample_folder)

pfs: List[PolicyFragment] = [
    PolicyFragment('AuthX-HR-Member', pf_authx_hr_member_xml, 'Authenticates and authorizes HR members.')
]

# 5) Define the Products

# HR Product with authentication policy, including authorization via a required claim check for HR member role
hr_product_xml = utils.read_policy_xml('hr_product.xml', {
    'jwt_signing_key': nb_helper.jwt_key_name, 
    'hr_member_role_id': 'HRMemberRoleId'
}, sample_folder)

hr_product_name = 'hr'
products: List[Product] = [
    Product(hr_product_name, 'Human Resources', 
            'Product for Human Resources APIs providing access to employee data, organizational structure, benefits information, and HR management services. Includes JWT-based authentication for HR members.', 
            'published', False, False, hr_product_xml)
]

# 6) Define the APIs and their operations and policies

# Policies
pol_hr_get = utils.read_policy_xml('hr_get.xml', sample_name = sample_folder)
pol_hr_post = utils.read_policy_xml('hr_post.xml', sample_name = sample_folder)
pol_require_product_xml = utils.read_policy_xml(REQUIRE_PRODUCT_XML_POLICY_PATH)

# Employees (HR)
hremployees_api_path = f'/{api_prefix}employees'
hremployees_get = GET_APIOperation('Gets the employees', pol_hr_get,)
hremployees_post = POST_APIOperation('Creates a new employee', pol_hr_post)
hremployees = API(f'{api_prefix}Employees', 'Employees Pro', hremployees_api_path, 'This is a Human Resources API for employee information', pol_require_product_xml, 
                  operations = [hremployees_get, hremployees_post], tags = tags, productNames = [hr_product_name], subscriptionRequired = False)

# Benefits (HR)
hrbenefits_api_path = f'/{api_prefix}benefits'
hrbenefits_get = GET_APIOperation('Gets employee benefits', pol_hr_get)
hrbenefits_post = POST_APIOperation('Creates employee benefits', pol_hr_post)
hrbenefits = API(f'{api_prefix}Benefits', 'Benefits Pro', hrbenefits_api_path, 'This is a Human Resources API for employee benefits', pol_require_product_xml, 
                 operations = [hrbenefits_get, hrbenefits_post], tags = tags, productNames = [hr_product_name], subscriptionRequired = False)

# APIs Array
apis: List[API] = [hremployees, hrbenefits]

utils.print_ok('Notebook initialized')

### 🚀 2. Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [ ]:
import utils

# 1) Define the Bicep parameters with serialized APIs
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]},
    'namedValues': {'value': [nv.to_dict() for nv in nvs]},
    'policyFragments': {'value': [pf.to_dict() for pf in pfs]},
    'products': {'value': [product.to_dict() for product in products]}
}

# 2) Deploy the bicep template
output = nb_helper.deploy_bicep(bicep_parameters)

if output.json_data:
    apim_name = output.get('apimServiceName', 'APIM Service Name')
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')

utils.print_ok('Deployment completed')

### ✅ 3. Verify API Request Success

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
import utils
from apimrequests import ApimRequests
from apimtesting import ApimTesting
from apimtypes import Role
from users import UserHelper
from authfactory import AuthFactory

tests = ApimTesting("AuthX-Pro Sample Tests", sample_folder, deployment)

# Preflight: Check if the infrastructure architecture deployment uses Azure Front Door. If so, assume that APIM is not directly accessible and use the Front Door URL instead.
endpoint_url = utils.test_url_preflight_check(deployment, rg_name, apim_gateway_url)

# 1) HR Administrator
# Create a JSON Web Token with a payload and sign it with the symmetric key from above.
encoded_jwt_token_hr_admin = AuthFactory.create_symmetric_jwt_token_for_user(UserHelper.get_user_by_role(Role.HR_ADMINISTRATOR), nb_helper.jwt_key_value)
print(f'\nJWT token for HR Admin:\n{encoded_jwt_token_hr_admin}')  # this value is used to call the APIs via APIM

# Set up an APIM requests object with the JWT token
reqsApimAdmin = ApimRequests(endpoint_url)
reqsApimAdmin.headers['Authorization'] = f'Bearer {encoded_jwt_token_hr_admin}'

# Call APIM
output = reqsApimAdmin.singleGet(hremployees_api_path, msg = 'Calling GET Employees API via API Management Gateway URL. Expect 200.')
tests.verify(output, 'Successful GET')

output = reqsApimAdmin.singlePost(hremployees_api_path, msg = 'Calling POST Employees API via API Management Gateway URL. Expect 200.')
tests.verify(output, 'Successful POST')

output = reqsApimAdmin.singleGet(hrbenefits_api_path, msg = 'Calling GET Benefits API via API Management Gateway URL. Expect 200.')
tests.verify(output, 'Successful GET')

output = reqsApimAdmin.singlePost(hrbenefits_api_path, msg = 'Calling POST Benefits API via API Management Gateway URL. Expect 200.')
tests.verify(output, 'Successful POST')

# 2) HR Associate
# Create a JSON Web Token with a payload and sign it with the symmetric key from above.
encoded_jwt_token_hr_associate = AuthFactory.create_symmetric_jwt_token_for_user(UserHelper.get_user_by_role(Role.HR_ASSOCIATE), nb_helper.jwt_key_value)
print(f'\nJWT token for HR Associate:\n{encoded_jwt_token_hr_associate}')  # this value is used to call the APIs via APIM

# Set up an APIM requests object with the JWT token
reqsApimAssociate = ApimRequests(endpoint_url)
reqsApimAssociate.headers['Authorization'] = f'Bearer {encoded_jwt_token_hr_associate}'

# Call APIM
output = reqsApimAssociate.singleGet(hremployees_api_path, msg = 'Calling GET Employees API via API Management Gateway URL. Expect 200.')
tests.verify(output, 'Successful GET')

output = reqsApimAssociate.singlePost(hremployees_api_path, msg = 'Calling POST Employees API via API Management Gateway URL. Expect 403.')
tests.verify(output, 'Access denied - no matching roles found')

output = reqsApimAssociate.singleGet(hrbenefits_api_path, msg = 'Calling GET Benefits API via API Management Gateway URL. Expect 200.')
tests.verify(output, 'Successful GET')

output = reqsApimAssociate.singlePost(hrbenefits_api_path, msg = 'Calling POST Benefits API via API Management Gateway URL. Expect 403.')
tests.verify(output, 'Access denied - no matching roles found')

tests.print_summary()

utils.print_ok('All done!')